# EDA and cleaning

- Where are the nulls? Naively drop them.
- What is the trend of the stock?

In [190]:
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, ShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA
from sklearn.linear_model import Lasso
from sklearn.ensemble import RandomForestRegressor

%matplotlib inline

In [37]:
apple = pd.read_csv("../data/sandp500/individual_stocks_5yr_TECHNICALS/AAPL_data.csv")
apple.head()

Unnamed: 0        Date   Open   High    Low  Close     Volume  Name  \
0           0  2012-08-13  89.06  90.00  89.04  90.00   69707463  AAPL   
1           1  2012-08-14  90.27  91.23  90.03  90.24   85041824  AAPL   
2           2  2012-08-15  90.19  90.57  89.68  90.12   64377278  AAPL   
3           3  2012-08-16  90.17  90.97  90.07  90.91   63694204  AAPL   
4           4  2012-08-17  91.43  92.60  91.26  92.59  110689894  AAPL   

   Acc/Dist_ROC_1  ADX_14_50    ...         RSI_6  RSI_12     STD_5  STD_20  \
0             0.0        NaN    ...           NaN     NaN       NaN     NaN   
1            -0.0        NaN    ...           NaN     NaN       NaN     NaN   
2            -0.0        NaN    ...           NaN     NaN       NaN     NaN   
3             0.0        NaN    ...           NaN     NaN       NaN     NaN   
4             0.0        NaN    ...      0.929766     NaN  1.075672     NaN   

     SO%d_5  SO%d_20      SO%k  Trix_5  Trix_20  TSI_25_13  
0       NaN      NaN  1.000000     0.0      0.0        NaN  
1       NaN      NaN  0.175000     NaN      NaN        NaN  
2       NaN      NaN  0.494382     NaN      NaN        NaN  
3  0.679983      NaN  0.933333     NaN      NaN        NaN  
4  0.799968      NaN  0.992537     NaN      NaN        NaN  

[5 rows x 50 columns]

In [38]:
apple["Date"] = pd.to_datetime(apple["Date"])
apple =  apple.set_index("Date")
apple = apple.drop("Unnamed: 0", axis = 1)

In [39]:
apple["Close"].plot()

Shift technicals down one so "tomorrow"s close can be predicted with yesterday's TIs

In [40]:
apple.iloc[:, 8:] = apple.iloc[:, 8:].shift()

In [48]:
# Drop column of all zeroes
apple = apple.dropna()
apple = apple.drop("Acc/Dist_ROC_1" , axis = 1)

In [137]:
y = apple["Close"]
X = apple.iloc[:, 6:]

# PCA on TIs

- How many dominant PCs are there?
    - Seems like between 5 and 10

In [123]:
X_standardized = StandardScaler().fit_transform(X)

pca = PCA()
pca.fit(X_standardized)
plt.plot(pca.explained_variance_ratio_)

Let's take a look at what the first few PCs are composed of. Yellow cells indicate contribution by feature

It would help a lot if I knew what the hell these TIs were

In [135]:
pc_composition = pd.DataFrame(pca.components_[0:5], index = ["PC1", "PC2", "PC3", "PC4", "PC5"], columns = X.columns).T


In [136]:
def highlight_bottom(s):
    '''
    highlight the max and min 3 in a Series yellow.
    '''
    is_blue = s.isin(s.sort_values()[:5])
    return ['background-color: aqua' if v else '' for v in is_blue]


def highlight_top(s):
    '''
    highlight the max and min 3 in a Series yellow.
    '''
    is_red = s.isin(s.sort_values(ascending = False)[:5])
    return ['background-color: crimson' if v else '' for v in is_red]

pc_composition.style.apply(highlight_bottom).apply(highlight_top)

# Baseline predictions

- How well can we predict today's closing price with yesterday's technicals? Which features are influential?

In [138]:
# calculate train split size
2 * apple.shape[0]  / 3

796.0

In [56]:
#chronological train test split

X_train, X_test = apple.iloc[:796, 6:], apple.iloc[796:, 6:]
y_train, y_test = apple.ix[:796, "Close"], apple.ix[796:, "Close"]



## Predicting tomorrow with yesterday's value

Before fitting anything let's see what $R^2$ you get by predicting tomorrow with yesterday's value

In [193]:
y_hat_shift = y_test.shift()[1:] 
y_test_shift = y_test[1:]

r2_score(y_hat_shift, y_test_shift)

0.99419287803255429

This is pretty telling. It is not difficult to predict tomorrow's closing price. What you need is to be able to predict far into the future, or this trading signal.

## Random Forest

In [91]:
rf = RandomForestRegressor(n_estimators = 10)
rf.fit(X_train, y_train)
print("Train score:", rf.score(X_train, y_train))
print("Test score:", rf.score(X_test, y_test))

Train score: 0.999089246704
Test score: 0.761171811665


Completely untuned model gets nearly 100% performance on the training set and 76% on the test set. So we have some over-ftting going on.

In [78]:
#technicals associated with closing price
apple.corr()["Close"].abs().sort_values(ascending = False)[4:].head(10)

PP        0.998090
R1        0.997967
S1        0.997956
R2        0.997269
S2        0.997253
EMA_5     0.996778
R3        0.996369
S3        0.996246
MA_5      0.996055
EMA_20    0.990233
Name: Close, dtype: float64

In [76]:
#feature importances
sorted(zip(X.columns, rf.feature_importances_), key = lambda x: abs(x[1]), reverse = True)[:10]

[('S1', 0.29998114100464257),
 ('R1', 0.21705884960783034),
 ('PP', 0.1982505624274063),
 ('EMA_20', 0.11494795115897972),
 ('MA_20', 0.074121183033869859),
 ('R3', 0.048031172332382022),
 ('R2', 0.025111469354456226),
 ('EMA_5', 0.011157941550139848),
 ('S2', 0.005674820811660577),
 ('MA_5', 0.0015132378363205668)]

Feature importances seem to agree with correlations between Close price and TIs

# Trying out a modeling Pipeline with PCA

In [177]:
rf_pipe_params = {"pca__n_components": np.arange(1,40, 5, dtype = int),
                  "model__n_estimators": [10, 50, 100]}

rf_pipe = Pipeline([("scaler", StandardScaler()),
                     ("pca", PCA()),
                     ("model", RandomForestRegressor())])



lasso_pipe = Pipeline([("scaler", StandardScaler()),
                     ("pca", PCA()),
                     ("model", Lasso())])

lasso_pipe_params = {"pca__n_components": np.arange(1,40, 5, dtype = int),
                  "model__alpha": np.logspace(-2, 3, 5)}


In [172]:
#Random forest pipeline
rf_gs = GridSearchCV(rf_pipe, rf_pipe_params,
                     cv = 3,
                     verbose = 1)
rf_gs.fit(X_train, y_train)

Fitting 3 folds for each of 24 candidates, totalling 72 fits


[Parallel(n_jobs=1)]: Done  72 out of  72 | elapsed:   25.7s finished


GridSearchCV(cv=3, error_score='raise',
       estimator=Pipeline(steps=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True)), ('pca', PCA(copy=True, iterated_power='auto', n_components=None, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('model', RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           m...timators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'pca__n_components': array([ 1,  6, 11, 16, 21, 26, 31, 36]), 'model__n_estimators': [10, 50, 100]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=1)

In [174]:
rf_gs.best_estimator_.fit(X_train, y_train)
rf_gs.best_estimator_.score(X_test, y_test)

0.65336949232114272

In [178]:
#Lasso pipeline
lasso_gs = GridSearchCV(lasso_pipe, lasso_pipe_params,
                     cv = 3,
                     verbose = 1)

lasso_gs.fit(X_train, y_train)

Fitting 3 folds for each of 40 candidates, totalling 120 fits


[Parallel(n_jobs=1)]: Done 120 out of 120 | elapsed:    1.6s finished


0.99237002026761423

In [185]:
best_lasso = lasso_gs.best_estimator_

best_lasso.score(X_test, y_test)

0.99237002026761423

# Thoughts & next steps

- Is modelling tomorow's closing price financially valuable? Or do you need to be predicting further into the future? Or predicing a trading signal?
    - I don't think so. Need to move onto deeper predictions, or predicting the trading signal.
- Need to write code for predicting more than one timestep into the future. Do this by iteratively updating TI predictors based on the predictions themselves. How does it do vs ARIMA?
    - Can I assume that being able to train models that predict deep into the future will translate to predicting trading signals?
- Will training on multiple time-series improve performance? Need to write a pipeline to clean and train on multiple stocks at once.
- Is data quality a concern? Do current TI's need to be tuned further? Should We be using other TIs need to refer to literature.
- How do you keep predictor features continually updated as time progresses? Need a pipeline for that